In [2]:
import pandas as pd
import dask as dd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import statsmodels.formula.api as smf
import statistics

from sklearn.linear_model import LogisticRegression

import statsmodels.api as sm
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod import families
from statsmodels.stats.outliers_influence import variance_inflation_factor

from scipy import stats

/usr/local/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [3]:
def basePrep(df):
	df['rndf'] = df['rndf'].map({"Yes": 1, "No": 0})
	df['pdynamic'] = df['pdynamic'].map({'new_entrant': 1, 'incumbent': 0})	
	df["SNATION_PARTAL".lower()] = df["SNATION_PARTAL".lower()].map({"Y": 1, "N": 0})
	df['cr_bor_part'] = df['cr_bor_part'].map({"Y": 1, "N": 0})
	df['saf'] = df['saf'].map({"Y": 1, "N": 0})
	df['mfgf'] = df['mfgf'].map({"Yes": 1, "No": 0})
	df['jvf'] = df['jvf'].map({'Yes': 1, 'No': 0})
	df['natc_n'] = df['natc'].astype('category').cat.codes
	
	cols = ['rndf', 'jvf','ddist_bin','ddist_int', 'ddist_abs', 'ddist_year',
		'avg_emp_pp', 'public_count', 'pdynamic', "SNATION_PARTAL".lower(), 
		'cr_bor_part', 'mfgf', 'avg_emp_pp_log', 'saf', 'natc_n']
	df = df[cols].dropna()
	
	return df

def variation(df):
	cols = ['rndf', 'ddist_bin','ddist_int', 'ddist_abs', 'ddist_year', 'avg_emp_pp', 'public_count', 
			"SNATION_PARTAL".lower(), 'cr_bor_part', 'mfgf', 'saf', 'natc_n']
	X = (df[cols]).astype(float)
	y = df['pdynamic'].astype(float)
	X_constant = sm.add_constant(X, prepend=False)

	logit_model = GLM(y, X_constant, family=families.Binomial())
	logit_results = logit_model.fit()
	print(logit_results.summary())

## Try all potential variables

In [4]:
df = pd.read_csv('./data/JV_data_dist.csv')
df = basePrep(df)

variation(df)

                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                  729
Model:                            GLM   Df Residuals:                      716
Model Family:                Binomial   Df Model:                           12
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -425.86
Date:                Fri, 17 Jun 2022   Deviance:                       851.71
Time:                        18:43:37   Pearson chi2:                     753.
No. Iterations:                    19   Pseudo R-squ. (CS):             0.1367
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
rndf               0.3038      0.318      0.

## Filter only R&D alliances

In [5]:
df2 = pd.read_csv('./data/JV_data_dist.csv')

df2 = df2[df2['rndf'] == "Yes"]
df2 = basePrep(df2)

variation(df2)

                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                   66
Model:                            GLM   Df Residuals:                       55
Model Family:                Binomial   Df Model:                           10
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -31.168
Date:                Fri, 17 Jun 2022   Deviance:                       62.336
Time:                        18:43:37   Pearson chi2:                     58.5
No. Iterations:                     5   Pseudo R-squ. (CS):             0.2259
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
rndf               3.2602      1.269      2.

## Filter for alliance SIC

In [6]:
df3 = pd.read_csv("./data/JV_data_dist.csv")

#Select alliances that at least part of their goal was manufacturing of cars
df3 = df3[df3['sic'].str.contains('3711') == True]
df3 = basePrep(df3)

variation(df3)

                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                  311
Model:                            GLM   Df Residuals:                      299
Model Family:                Binomial   Df Model:                           11
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -184.74
Date:                Fri, 17 Jun 2022   Deviance:                       369.48
Time:                        18:43:38   Pearson chi2:                     342.
No. Iterations:                     4   Pseudo R-squ. (CS):             0.1716
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
rndf              -0.3437      0.697     -0.

## Filter for same nation participants and alliance

In [7]:
df4 = pd.read_csv("./data/JV_data_dist.csv")

#Select alliances that at least part of their goal was manufacturing of cars
df4 = df4[df4['snation_partal'] == "Y"]
df4 = basePrep(df4)

variation(df4)

                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                  154
Model:                            GLM   Df Residuals:                      143
Model Family:                Binomial   Df Model:                           10
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -71.551
Date:                Fri, 17 Jun 2022   Deviance:                       143.10
Time:                        18:43:38   Pearson chi2:                     178.
No. Iterations:                     5   Pseudo R-squ. (CS):             0.1041
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
rndf              -0.2858      0.711     -0.

## Filter for Manufacturing agreement flags

In [8]:
df5 = pd.read_csv("./data/JV_data_dist.csv")

#Select alliances that at least part of their goal was manufacturing of cars
df5 = df5[df5['mfgf'] == "Yes"]
df5 = basePrep(df5)

variation(df5)

                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                  481
Model:                            GLM   Df Residuals:                      470
Model Family:                Binomial   Df Model:                           10
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -288.41
Date:                Fri, 17 Jun 2022   Deviance:                       576.83
Time:                        18:43:38   Pearson chi2:                     499.
No. Iterations:                     4   Pseudo R-squ. (CS):             0.1454
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
rndf               0.7745      0.500      1.

In [9]:
df6 = pd.read_csv("./data/JV_data_dist.csv")

#Select alliances that at least part of their goal was manufacturing of cars
df6 = df6[df6['mfgf'] == "Yes"]
df6 = df6[df6['sic'].str.contains('3711') == True]
df6 = basePrep(df6)

variation(df6)

                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                  267
Model:                            GLM   Df Residuals:                      256
Model Family:                Binomial   Df Model:                           10
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -158.93
Date:                Fri, 17 Jun 2022   Deviance:                       317.87
Time:                        18:43:38   Pearson chi2:                     296.
No. Iterations:                     4   Pseudo R-squ. (CS):             0.1693
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
rndf              -0.1081      0.821     -0.

In [10]:
df7 = pd.read_csv("./data/JV_data_dist.csv")

#Select alliances that at least part of their goal was manufacturing of cars
df7 = df7[df7['mfgf'] == "Yes"]
df7 = df7[df7['sic'].str.contains('3711') == True]
df7 = df7[df7['public_count'] >= 1]
df7 = df7[df7['nump'] == 2]
df7 = basePrep(df7)

variation(df7)

                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                  202
Model:                            GLM   Df Residuals:                      191
Model Family:                Binomial   Df Model:                           10
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -115.92
Date:                Fri, 17 Jun 2022   Deviance:                       231.84
Time:                        18:43:39   Pearson chi2:                     218.
No. Iterations:                     4   Pseudo R-squ. (CS):             0.1969
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
rndf              -0.1598      1.311     -0.

## Filter for single country

In [11]:
df8 = pd.read_csv("./data/JV_data_dist.csv")

#Select alliances that at least part of their goal was manufacturing of cars
df8 = df8[df8['natc'] == "IN"]

df8 = basePrep(df8)

variation(df8)

                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                   76
Model:                            GLM   Df Residuals:                       65
Model Family:                Binomial   Df Model:                           10
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -30.312
Date:                Fri, 17 Jun 2022   Deviance:                       60.624
Time:                        18:43:39   Pearson chi2:                     855.
No. Iterations:                     6   Pseudo R-squ. (CS):             0.3170
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
rndf              -1.4303      1.774     -0.

## Test for final H1

In [17]:
dfH = pd.read_csv("./data/JV_data_dist.csv")

dfH = dfH[dfH['mfgf'] == "Yes"]
dfH = dfH[dfH['sic'].str.contains('3711') == True]
dfH = dfH[dfH['public_count'] >= 1]
# dfH = dfH[dfH['nump'] == 2]
# dfH = dfH[dfH['avg_emp_pp'] >= 1]
dfH = basePrep(dfH)

cols = ['ddist_year']
X = (dfH[cols]).astype(float)
y = dfH['pdynamic'].astype(float)
X_constant = sm.add_constant(X, prepend=False)

logit_model = GLM(y, X_constant, family=families.Binomial())
logit_results = logit_model.fit()
print(logit_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                  239
Model:                            GLM   Df Residuals:                      237
Model Family:                Binomial   Df Model:                            1
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -155.19
Date:                Fri, 17 Jun 2022   Deviance:                       310.38
Time:                        18:45:10   Pearson chi2:                     239.
No. Iterations:                     4   Pseudo R-squ. (CS):            0.05905
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ddist_year     0.1510      0.041      3.703      0.0

In [26]:
dfH['ddist_year']

3     -5
5     -5
6     -5
8     -5
10    -5
      ..
977    5
978    5
982    5
985    5
986    5
Name: ddist_year, Length: 239, dtype: int64

## Test for final H2

In [32]:
dfH2 = pd.read_csv("./data/JV_data_dist.csv")

dfH2 = dfH2[dfH2['mfgf'] == "Yes"]
dfH2 = dfH2[dfH2['sic'].str.contains('3711') == True]
dfH2 = dfH2[dfH2['public_count'] >= 1]
dfH2 = dfH2[dfH2['nump'] == 2]
dfH2 = dfH2[dfH2['emp_min'] >= 100]


dfH2 = basePrep(dfH2)


cols = ['ddist_year', 'public_count', 'rndf', 'snation_partal', 'avg_emp_pp_log', 'pdynamic']
X = (dfH2[cols]).astype(float)
y = dfH2['jvf'].astype(float)
X_constant = sm.add_constant(X, prepend=False)

logit_model = GLM(y, X_constant, family=families.Binomial())
logit_results = logit_model.fit()
print(logit_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                    jvf   No. Observations:                  195
Model:                            GLM   Df Residuals:                      188
Model Family:                Binomial   Df Model:                            6
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -104.66
Date:                Fri, 17 Jun 2022   Deviance:                       209.31
Time:                        19:11:41   Pearson chi2:                     196.
No. Iterations:                     4   Pseudo R-squ. (CS):             0.1098
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
ddist_year         0.1665      0.053      3.

In [20]:
dfH2['ddist_year']

3     -5
8     -5
10    -5
41    -5
61    -5
      ..
949    5
950    5
964    5
985    5
986    5
Name: ddist_year, Length: 100, dtype: int64